In [ ]:
# Ajax加载

In [ ]:
## (1) 发送请求
## (2) 解析内容
## (3) 渲染网页

In [ ]:
### 现在大多数网页利用ajax原因
    使用Ajax的主要原因  
　　1、通过适当的Ajax应用达到更好的用户体验；  
　　2、把以前的一些服务器负担的工作转嫁到客户端，利于客户端闲置的处理能力来处理，减轻服务器和带宽的负担，从而达到节约ISP的空间及带宽租用成本的目的。

In [ ]:
from urllib.parse import urlencode
import requests
from pyquery import PyQuery as pq
base_url = 'https://m.weibo.cn/api/container/getIndex?'

headers ={
    'Host' : 'm.weibo.cn',
    'Referer':'https://m.weibo.cn/u/2830678474',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36',
    'X-Requested-With':'XMLHttpRequest',
}

def get_page(since_id):
    params = {
        'type':'uid',
        'value':'2830678474',
        'containerid':'1005052830678474',
    }
    if since_id !=0:
       params['since_id'] = since_id
    url = base_url + urlencode(params)
    try :
        response = requests.get(url, headers = headers)
        if response.status_code == 200:
            return response.json()
    except requests.ConnectionError as e:
        print('Error', e.args)

def parse_page(json):
    if json :
        items = json.get('data').get('cards')
        for item in items:
            item = item.get('mblog')
            weibo = {}
            weibo['id'] = item.get('id')
            weibo['text'] = pq(item.get('text')).text()
            weibo['attitudes'] = item.get('attitudes_count')
            weibo['comments'] = item.get('comments_count')
            weibo['reposts'] = items.get('reposts_count')
            yield weibo

if __name__ == '__main__':
    since_id = 0
    for page in range(1,max_page+1):
        json = get_page(since_id)
        since_id = json.get('data').get('cardlistinfo').get('since_id')
        results = parse_page(json)
        for result in results:
            with open('weibocui.json','w') as file:
                file.write(json.dumps(result, indent = 2))
            print(result)